In [50]:
import numpy as np
import xarray as xr

# Load the ERA5 data
ds = xr.open_dataset('/Users/joaojesus/Desktop/final_year_proj/samples/jan-march-2009-geopotential-10-100hpa-30-90N.nc')

# Subset the data to the desired sub-region and pressure levels
ds_sub = ds.sel(latitude=slice(90,30), longitude=slice(-180,180), level=[5,7,10,20])




In [51]:

ds['z'][0][0][0][0].values

array(317594.06, dtype=float32)

In [52]:
# Loop over each day and time and calculate the mean geopotential height
means = []
for day_time in ds_sub.time:
        da = ds_sub.sel(time=day_time)['z'].mean(dim=['latitude', 'longitude'])
        means.append(da)

# Calculate the overall mean geopotential height
mean_z = np.mean(means)

In [53]:
# Loop over each day and time and calculate the geopotential height perturbations
perturbations = []
for day_time in ds_sub.time:
        da = ds_sub.sel(time=day_time)['z']
        pert = (da - mean_z) / 9.80665  # Divide by g to convert to units of meters
        perturbations.append(pert)

# Concatenate the perturbation data into a single array
perturbations = xr.concat(perturbations, dim='time')

# Save the perturbation data to a NetCDF file
perturbations.to_netcdf('perturbation_data.nc')